In [ ]:
# Import necessary libraries
from flask import Flask, jsonify, request
import requests
import os


In [ ]:
# Initialize Flask application
app = Flask(__name__)

# GitHub API token (You may need to set this as an environment variable)
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

# GitHub API base URL
GITHUB_API_URL = 'https://api.github.com'


In [ ]:
# Helper function to get issues from GitHub
def get_github_issues(owner, repo, page, per_page):
    headers = {
        'Authorization': f'token {GITHUB_TOKEN}',
        'Accept': 'application/vnd.github.v3+json'
    }
    params = {
        'q': f'repo:{owner}/{repo} is:issue is:closed label:bug linked:pr',
        'page': page,
        'per_page': per_page
    }
    response = requests.get(f'{GITHUB_API_URL}/search/issues', headers=headers, params=params)
    response.raise_for_status()
    return response.json()


In [ ]:
# Route to fetch issues from GitHub
@app.route('/issues', methods=['GET'])
def issues():
    owner = request.args.get('owner')
    repo = request.args.get('repo')
    page = request.args.get('page', 1, type=int)
    per_page = request.args.get('per_page', 10, type=int)
    if not owner or not repo:
        return jsonify({'error': 'Owner and repo parameters are required'}), 400
    try:
        issues_data = get_github_issues(owner, repo, page, per_page)
        issues_list = []
        for item in issues_data.get('items', []):
            issue = {
                'id': item['id'],
                'title': item['title'],
                'body': item['body'],
                'url': item['html_url']
            }
            issues_list.append(issue)
        return jsonify(issues_list)
    except requests.exceptions.RequestException as e:
        return jsonify({'error': str(e)}), 500


In [ ]:
# Run the Flask application
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
